In [1]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [33]:
retail = pd.read_csv('C:/Users/79295/Desktop/GITHUB/Analytical-practice/datasets/5_data.csv.zip', compression = 'zip', encoding = 'ISO-8859-1')

In [34]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [35]:
# Сконцентрируемся на аналитике и разберем кейс с рекламными данными. 
# Освоим сводные таблицы и методы для работы с пропущенными значениями.




#### Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:
- InvoiceNo — номер транзакции
- StockCode — код товара
- Description — описание товара
- Quantity — количество единиц товара, добавленных в заказ
- InvoiceDate — дата транзакции 
- UnitPrice — цена за единицу товара
- CustomerID — id клиента
- Country — страна, где проживает клиент

In [36]:
# Импортируйте пандас и прочитайте данные с кодировкой ISO-8859-1. Запишите полученный датафрейм в retail, 
# а названия колонок сохраните в переменную retail_columns.

In [37]:
retail_columns = retail.columns

In [38]:
# Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. 
# Если они есть, то удалите их из retail.

In [39]:
retail.nunique()

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

In [40]:
retail.shape

(541909, 8)

In [41]:
retail.duplicated().sum()

5268

In [42]:
retail = retail.drop_duplicates()

In [43]:
retail.shape

(536641, 8)

In [44]:
# Данные содержат в себе записи как об успешных транзакциях, так и об отмененных. 
# Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled).

#Сколько всего транзакций отменили пользователи? Считайте, что каждая строка - это отдельная транзакция.

In [45]:
retail.InvoiceNo.str.startswith("C").sum()

9251

In [46]:
# Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0.
# В качестве ответа укажите число оставшихся строк.

In [47]:
retail[retail.Quantity > 0].shape[0]

526054

In [48]:
# Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). 
# Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. 
# Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).

In [49]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [61]:
quantile80 = retail[retail.Country == 'Germany'].groupby('CustomerID',as_index=False) \
    .agg({"InvoiceNo":"count"}) \
    .rename(columns={"InvoiceNo":"total_transaction"}) \
    .total_transaction.quantile(0.8)
quantile80

164.0

In [65]:
germany_top = retail[retail.Country == 'Germany'].groupby('CustomerID',as_index=False) \
    .agg({"InvoiceNo":"count"}) \
    .rename(columns={"InvoiceNo":"total_transaction"}) \
    .query('total_transaction > 164').CustomerID
germany_top

3     12471.0
4     12472.0
5     12473.0
6     12474.0
8     12476.0
9     12477.0
12    12481.0
20    12517.0
37    12569.0
53    12621.0
57    12626.0
58    12627.0
70    12662.0
72    12668.0
78    12705.0
79    12708.0
80    12709.0
84    12720.0
91    13815.0
Name: CustomerID, dtype: float64

In [66]:
# Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения 
# и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.

In [75]:
top_retail_germany = retail[retail.CustomerID.isin(germany_top)]
top_retail_germany

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany
...,...,...,...,...,...,...,...,...
541726,581570,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,12/9/2011 11:59,4.95,12662.0,Germany
541727,581570,23077,DOUGHNUT LIP GLOSS,20,12/9/2011 11:59,1.25,12662.0,Germany
541728,581570,20750,RED RETROSPOT MINI CASES,2,12/9/2011 11:59,7.95,12662.0,Germany
541729,581570,22505,MEMO BOARD COTTAGE DESIGN,4,12/9/2011 11:59,4.95,12662.0,Germany


In [76]:
# Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST?

# Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

In [87]:
top_retail_germany.groupby('StockCode', as_index=False) \
    .agg({"InvoiceNo":"count"}) \
    .rename(columns={"InvoiceNo":"total"}) \
    .sort_values(by = 'total', ascending=False) \
    .query('StockCode != "POST"').head(1)

,StockCode,total
464,22326,75


In [88]:
# Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.

In [89]:
retail['Revenue'] = retail['Quantity'] * retail['UnitPrice']
retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,16.60


In [91]:
# Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. В качестве ответа укажите топ-5 (InvoiceNo) 
# по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)



In [127]:
top_5_invoice = retail.groupby('InvoiceNo',as_index=False) \
    .agg({"Revenue":"sum"}) \
    .rename(columns={"Revenue":"total_revenue"}) \
    .sort_values(by='total_revenue', ascending=False) \
    .head(5).InvoiceNo
top_5_invoice

22025    581483
2303     541431
18776    574941
19468    576365
9741     556444
Name: InvoiceNo, dtype: object

In [128]:
top_invoice = []

In [129]:
for i in top_5_invoice:
    top_invoice.append(i)
top_invoice  

        
    

['581483', '541431', '574941', '576365', '556444']